In [122]:
import numpy as np
import pickle
import math
import random

from qap import *
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn import preprocessing

In [114]:
def parse(fileName):
    with open(fileName, "r") as f:
        lines = list(filter(None, (line.rstrip() for line in f)))
        n = int(lines[0])
        a = sum([list(map(int, line.split())) for line in lines[1:]], [])
        d = np.array(a[:n**2]).reshape(n, n).astype(int)
        f = np.array(a[n**2:]).reshape(n, n).astype(int)
    return n, d, f

In [115]:
fileName = "./data/Large/lipa90a.dat"
n, d, f = parse(fileName)
# inst = QAP(d, f, True)

In [119]:
def cal_product(n, d, f, sta):
    if sta == 'mean':
        f_m = [np.mean(f[i]) for i in range(n)]
        d_m = [np.mean(d[i]) for i in range(n)]
    elif sta == 'median':
        f_m = [np.median(f[i]) for i in range(n)]
        d_m = [np.median(d[i]) for i in range(n)]
    else:
        f_m = [np.sum(f[i]) for i in range(n)]
        d_m = [np.sum(d[i]) for i in range(n)]
    f_order = list(np.argsort(f_m))[::-1]
    d_order = np.argsort(d_m)
    products = []
    pairs = []
    for i in range(n):
        products.append(f_m[f_order[i]] * d_m[d_order[i]])
        pairs.append((f_order[i], d_order[i]))
    return products, pairs
    
def create_cluster(n, d, f, sta, n_clusters):
    products, pairs = cal_product(n, d, f, sta)
    X = np.array(products).reshape(n, 1)
#     clustering = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
    clustering = AgglomerativeClustering(n_clusters=n_clusters).fit(X)
    labels = clustering.labels_
    print(labels)
    clusters = []
    for i in range(n_clusters):
        index = np.where(labels==i)[0].tolist()
        if len(index) > 40:
            # k = len(index) // 30
            k = math.ceil(len(index) / 30)
            for j in range(k):
                if j != k - 1:
                    sub_index = index[j*30:(j+1)*30]
                    clusters.append([pairs[x] for x in sub_index])
                else:
                    sub_index = index[j*30:]
                    clusters.append([pairs[x] for x in sub_index])
        else:
            clusters.append([pairs[x] for x in index])
    return clusters

In [117]:
products, pairs = cal_product(n, d, f, 'sum')
print(pairs)

[(88, 0), (10, 64), (7, 63), (3, 62), (5, 61), (24, 60), (87, 59), (31, 58), (30, 57), (19, 65), (14, 56), (48, 54), (50, 53), (45, 52), (4, 51), (89, 50), (17, 49), (84, 48), (8, 47), (29, 55), (9, 66), (79, 67), (57, 68), (51, 87), (58, 86), (68, 85), (18, 84), (12, 83), (55, 82), (20, 81), (64, 80), (59, 79), (69, 78), (46, 77), (26, 76), (0, 75), (80, 74), (72, 73), (27, 72), (13, 71), (47, 70), (40, 69), (52, 46), (16, 45), (28, 44), (36, 43), (37, 19), (32, 18), (54, 17), (73, 16), (83, 15), (78, 14), (77, 13), (23, 12), (15, 11), (38, 10), (41, 9), (33, 8), (6, 7), (71, 6), (25, 5), (60, 4), (42, 3), (74, 2), (82, 1), (43, 20), (22, 88), (61, 21), (2, 23), (65, 42), (85, 41), (63, 40), (81, 39), (49, 38), (53, 37), (56, 36), (1, 35), (75, 34), (44, 33), (67, 32), (35, 31), (70, 30), (34, 29), (62, 28), (39, 27), (76, 26), (11, 25), (21, 24), (66, 22), (86, 89)]


In [118]:
clusters = create_cluster(n, d, f, 'sum', 5)
print(clusters)

[3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 4 4 4]
[[(9, 66), (79, 67), (57, 68), (51, 87), (58, 86), (68, 85), (18, 84), (12, 83), (55, 82), (20, 81), (64, 80), (59, 79), (69, 78), (46, 77), (26, 76), (0, 75), (80, 74), (72, 73), (27, 72), (13, 71), (47, 70), (40, 69), (52, 46), (16, 45), (28, 44), (36, 43), (37, 19), (32, 18), (54, 17), (73, 16), (83, 15), (78, 14), (77, 13), (23, 12), (15, 11), (38, 10), (41, 9), (33, 8), (6, 7), (71, 6)], [(25, 5), (60, 4), (42, 3), (74, 2), (82, 1), (43, 20), (22, 88), (61, 21), (2, 23), (65, 42), (85, 41), (63, 40), (81, 39), (49, 38), (53, 37), (56, 36), (1, 35), (75, 34), (44, 33), (67, 32), (35, 31), (70, 30), (34, 29), (62, 28), (39, 27), (76, 26), (11, 25)], [(10, 64), (7, 63), (3, 62), (5, 61), (24, 60), (87, 59), (31, 58), (30, 57), (19, 65), (14, 56), (48, 54), (50, 53), (45, 52), (4, 51), (89, 50), (17, 49)

In [120]:
clusters = create_cluster(n, d, f, 'sum', 5)
print(clusters)

[4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3]
[[(78, 14), (77, 13), (23, 12), (15, 11), (38, 10), (41, 9), (33, 8), (6, 7), (71, 6), (25, 5), (60, 4), (42, 3), (74, 2), (82, 1), (43, 20), (22, 88), (61, 21), (2, 23), (65, 42), (85, 41), (63, 40), (81, 39), (49, 38), (53, 37), (56, 36), (1, 35), (75, 34), (44, 33), (67, 32), (35, 31), (70, 30), (34, 29), (62, 28), (39, 27), (76, 26), (11, 25)], [(10, 64), (7, 63), (3, 62), (5, 61), (24, 60), (87, 59), (31, 58), (30, 57), (19, 65), (14, 56), (48, 54), (50, 53), (45, 52), (4, 51), (89, 50), (17, 49), (84, 48), (8, 47), (29, 55)], [(9, 66), (79, 67), (57, 68), (51, 87), (58, 86), (68, 85), (18, 84), (12, 83), (55, 82), (20, 81), (64, 80), (59, 79), (69, 78), (46, 77), (26, 76), (0, 75), (80, 74), (72, 73), (27, 72), (13, 71), (47, 70), (40, 69), (52, 46), (16, 45), (28, 44), (36, 43), (37, 19), (32, 18),

# multi-d clustering

In [107]:
def cal_product_(n, d, f, sta):
    if sta == 'mean':
        f_m = [np.mean(f[i]) for i in range(n)]
        d_m = [np.mean(d[i]) for i in range(n)]
    elif sta == 'median':
        f_m = [np.median(f[i]) for i in range(n)]
        d_m = [np.median(d[i]) for i in range(n)]
    else:
        f_m = [np.sum(f[i]) for i in range(n)]
        d_m = [np.sum(d[i]) for i in range(n)]
    f_order = list(np.argsort(f_m))[::-1]
    d_order = np.argsort(d_m)
    products = []
    pairs = []
    for i in range(n):
        products.append(f_m[f_order[i]] * d_m[d_order[i]])
        pairs.append((f_order[i], d_order[i]))
    f_std = [np.std(f[i]) for i in range(n)]
    d_std = [np.std(d[i]) for i in range(n)]
    f_median = [np.median(f[i]) for i in range(n)]
    d_median = [np.median(d[i]) for i in range(n)]
    return np.array(f_std)[f_order], np.array(d_std)[d_order], np.array(f_median)[f_order], np.array(d_median)[d_order], products, pairs
    
def create_cluster_(n, d, f, sta, n_clusters):
    f_std, d_std, f_median, d_median, products, pairs = cal_product_(n, d, f, sta)
    X = np.array([f_std, d_std, f_median, d_median, products]).reshape(5, n).T
    scaler = preprocessing.StandardScaler().fit(X)
    X_scaled = scaler.transform(X)
#     clustering = KMeans(n_clusters=n_clusters, random_state=0).fit(X_scaled)
    clustering = AgglomerativeClustering(n_clusters=n_clusters).fit(X_scaled)
    labels = clustering.labels_
    print(labels)
    clusters = []
    for i in range(n_clusters):
        index = np.where(labels==i)[0].tolist()
        if len(index) > 40:
            # k = len(index) // 30
            k = math.ceil(len(index) / 30)
            for j in range(k):
                if j != k - 1:
                    sub_index = index[j*30:(j+1)*30]
                    clusters.append([pairs[x] for x in sub_index])
                else:
                    sub_index = index[j*30:]
                    clusters.append([pairs[x] for x in sub_index])
        else:
            clusters.append([pairs[x] for x in index])
    return clusters

In [105]:
f_std, d_std, f_median, d_median, products, pairs = cal_product_(n, d, f, 'sum')
print(products)

[418478, 399076, 389553, 388574, 387061, 387061, 384925, 384035, 382166, 380831, 380297, 378695, 378606, 378517, 378250, 376203, 376203, 375402, 375224, 374779, 372554, 372287, 370685, 370418, 369973, 369528, 368549, 368460, 368371, 368193, 368104, 368015, 368015, 367570, 367481, 366502, 366324, 366146, 365968, 365879, 365701, 365256, 365078, 364989, 364900, 364277, 363387, 362942, 361696, 360984, 360806, 359738, 359738, 359649, 359382, 359115, 358670, 358403, 358314, 358047, 356890, 356178, 356000, 355555, 354665, 354220, 353508, 353063, 352351, 352351, 352173, 351995, 351283, 350571, 349325, 349236, 348880, 348613, 346833, 346388, 346121, 345854, 345765, 345676, 344341, 343184, 340247, 332682, 328232, 320222]


In [106]:
clusters = create_cluster_(n, d, f, 'sum', 5)
print(clusters)

[4 4 4 4 1 4 4 4 4 2 2 4 4 4 2 1 2 1 2 2 2 2 1 1 2 2 2 1 1 2 2 2 1 2 1 2 1
 2 2 2 1 1 2 2 2 1 1 2 2 1 1 2 2 1 1 2 1 2 2 1 1 1 1 1 0 2 2 0 1 3 0 1 0 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[[(82, 1), (61, 21), (85, 41), (81, 39), (53, 37), (56, 36), (1, 35), (75, 34), (44, 33), (67, 32), (70, 30), (34, 29), (62, 28), (39, 27), (76, 26), (11, 25), (21, 24), (66, 22), (86, 89)], [(5, 61), (89, 50), (84, 48), (57, 68), (51, 87), (12, 83), (55, 82), (69, 78), (26, 76), (80, 74), (47, 70), (40, 69), (36, 43), (37, 19), (73, 16), (83, 15), (23, 12), (15, 11), (41, 9), (71, 6), (25, 5), (60, 4), (42, 3), (74, 2), (2, 23), (63, 40), (49, 38), (35, 31)], [(19, 65), (14, 56), (4, 51), (17, 49), (8, 47), (29, 55), (9, 66), (79, 67), (58, 86), (68, 85), (18, 84), (20, 81), (64, 80), (59, 79), (46, 77), (0, 75), (72, 73), (27, 72), (13, 71), (52, 46), (16, 45), (28, 44), (32, 18), (54, 17), (78, 14), (77, 13), (38, 10), (33, 8), (6, 7), (43, 20), (22, 88)], [(65, 42)], [(88, 0), (10, 64), (7, 63), (3, 6

In [108]:
f_std, d_std, f_median, d_median, products, pairs = cal_product_(n, d, f, 'sum')
print(products)
clusters = create_cluster_(n, d, f, 'sum', 5)
print(clusters)

[418478, 399076, 389553, 388574, 387061, 387061, 384925, 384035, 382166, 380831, 380297, 378695, 378606, 378517, 378250, 376203, 376203, 375402, 375224, 374779, 372554, 372287, 370685, 370418, 369973, 369528, 368549, 368460, 368371, 368193, 368104, 368015, 368015, 367570, 367481, 366502, 366324, 366146, 365968, 365879, 365701, 365256, 365078, 364989, 364900, 364277, 363387, 362942, 361696, 360984, 360806, 359738, 359738, 359649, 359382, 359115, 358670, 358403, 358314, 358047, 356890, 356178, 356000, 355555, 354665, 354220, 353508, 353063, 352351, 352351, 352173, 351995, 351283, 350571, 349325, 349236, 348880, 348613, 346833, 346388, 346121, 345854, 345765, 345676, 344341, 343184, 340247, 332682, 328232, 320222]
[2 2 2 2 0 2 2 2 2 4 0 4 2 2 4 0 4 0 4 4 4 4 0 0 0 4 4 0 0 4 4 4 0 4 0 4 0
 4 4 4 0 0 4 4 4 0 0 4 4 0 0 4 4 0 0 4 0 4 4 0 0 0 0 0 1 4 4 1 0 3 1 0 1 0
 0 1 1 0 1 0 0 1 0 1 1 0 0 0 1 1]
[[(5, 61), (14, 56), (89, 50), (84, 48), (57, 68), (51, 87), (58, 86), (12, 83), (55, 82), (69,